In [1]:
import os
import matplotlib.pyplot as plt
import cv2
import numpy as np

gender_folder = r"C:\Users\agboo\Downloads\Gender_Classification"
gender_class = ["women","men"]

# we will use this to traverse our features and append them along with their classes
# we will make every picture 100 X 120  this is due to most pictures being selfies / and or full body
row = 120
col = 100
feature = []
def training():
    for gender in range(2):
        path = os.path.join(gender_folder, gender_class[gender])
        # creating flag we will classify with either 0 or 1
        flag = gender_class.index(gender_class[gender]) 
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_COLOR)
                new_img = cv2.resize(img_array,(row,col))
                feature.append([new_img,flag])
            except:
                print(" Error: Corrupted File :( ")
training()
print(len(feature))

# import random so we can shuffle data
import random
random.shuffle(feature)

X = []
Y = []

for feat , ident in feature:
    X.append(feat)
    Y.append(ident)
X = np.array(X).reshape(-1,row,col,3) # features
Y = np.array(Y) # flags


## now we will save the data as pickle files to be loaded into the model
import pickle 
X_out = open("Feat.pickle","wb") # open file for data to then be dumped
pickle.dump(X, X_out)
X_out.close()

Y_out = open("Ident.pickle","wb") # the same is done for the flag list
pickle.dump(Y, Y_out)
Y_out.close()

#### the data will be loaded using rb (read binary when we start to construct the model




    
    

    

    

    
    
    

 Error: Corrupted File :( 
 Error: Corrupted File :( 
 Error: Corrupted File :( 
 Error: Corrupted File :( 
1581


In [17]:
import tensorflow as tf
import tensorflow.keras.models
from tensorflow.keras.layers import Conv2D , Flatten , Dense , Activation , MaxPooling2D , Dropout
from tensorflow.keras.callbacks import TensorBoard


import time
import pickle

X = pickle.load(open("Feat.pickle","rb"))
Y = pickle.load(open("Ident.pickle","rb"))

X = X/255.0 # this will normalize the feature array


def pretty(node):
    while node:
        print(str(node.val) + "->", end ='')
        node = node.next
    if node:
        print(node.val)
    else:
        print(None)
        
        

        
class ConvNode(keras.Model): # class for reshape node
    def __init__(self, val = 0 , next = None):
        super().__init__()
        self.val = val
        self.next = next
        self.conv = keras.layers.Conv2D(128,(3,3), input_shape = X.shape[1:])
        self.activation = keras.layers.Activation("relu")
        self.pool = keras.layers.MaxPooling2D(pool_size = (2,2))
        # so now this initial linked list has attributes of what would be a the beggining of our model sequentially

class HiddenNode(keras.Model): # class for hidden layers most likely will be the longest
    def __init__(self,val = 0 , next = None):
        super().__init__()
        self.val = val
        self.next = next
        self.conv = keras.layers.Conv2D(128,(3,3))
        self.activation = keras.layers.Activation("relu")
        self.pool = keras.layers.MaxPooling2D(pool_size = (2,2))

# this is the bridge between maxpooling and fully connected dense layers

class FlattenNode(keras.Model): # we need only one flatten node
    def __init__(self,val = 0 ,next = None):
        super().__init__()
        self.val = val
        self.next = next
        self.flat = keras.layers.Flatten()

class DenseNode(keras.Model): # this can also be k long , and will have to be fully connected so we will implement this with out call function
    def __init__(self,val = 0, next = None):
        super().__init__()
        self.val = val
        self.next = next
        self.dense1 = keras.layers.Dense(48,activation = "relu")
        self.dense2 = keras.layers.Dense(56,activation = "sigmoid")
    def call(self,inputs):
        x = self.dense1(inputs)
        return self.dense2(x)
    # since this is fully connected we need a call class to connect them
          
class OutputNode(keras.Model): # we need only one output node
    def __init__(self,val = 0 , next = None):
        super().__init__()
        self.val = val
        self.next = next
        self.output = keras.layers.Dense(2, activation = "softmax") # because we have two classes
    def call(self):
        return self.output
        
# the linked list will at least be 5 nodes long
def create_Model(hidden_value , dense_value):
    count = 0 # to help us control the node numbers
    cur = ConvNode(count) # conv
    count += 1
    dummy = cur

    for conv in range(hidden_value): # creating amount of 4 X hidden_value layers
        cur.next = HiddenNode(count)
        count += 1
        cur = cur.next
    cur.next = FlattenNode(count)  # flatten layer
    count += 1
    cur = cur.next
    for dense in range(dense_value): # dense layers
        cur.next = DenseNode(count).call(2)
        count += 1
        cur = cur.next
    cur.next = OutputNode(count).call() # output layer
    count += 1
    cur = cur.next
    dummy.compile(loss = "binary_crossentropy", optimizer = "adam" , metrics = ["accuracy"])
    dummy.fit(X,Y, batch_size = 32, epochs = 32, validation_split = 0.2)
    return pretty(dummy)
create_Model(2,2) # controling hidden and dense layers

# so now we have created the model within the linked list

# 7/23/24 You need pass tensors as arguments
# search up tensors 
# read more documentation on tensors and connecting layers





ValueError: Only input tensors may be passed as positional arguments. The following argument value should be passed as a keyword argument: 2 (of type <class 'int'>)